# Stressed VaR calibration with atoti

## A quick introduction

This jupyter notebook will demonstrate how atoti™ community edition can help to analyse and explore **Value-at-Risk** across observation windows. This tool can be particularly useful for clients looking to **recalibrate the period** of significant stress relevant to their portfolio, for instance, from the **Global Financial Crisis of 2008** to the **Coronavirus Market Crash of 2020**. 

Read more about the use case in this blog post: [Stressed VaR calibration](#).

We will focus on the step-by-step implementation of an analytical cube for the rolling VaR use case in atoti.


# Importing libraries

We will be using atoti and pandas today.

In [1]:
import atoti as tt
import pandas as pd

Welcome to atoti 0.4.0.20200519221045!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


Each PnL value in the returned vector corresponds to a scenario at the same index position.

# Sample data overview

For the purposes of this demonstration, we'll use a toy data model that can be adjusted for a real-life use case. The datastores that we'll need include:

- positions: books, stock tickers and positions market values in USD (long and short positions),
- historical returns: vectors of stock prices and daily changes, we will be multiplying market values by stock price changes to obtain a simulated historical P&L for the demonstration. The first two data stores can be replaced with full revaluation P&L vectors from a risk system.

# Creating an atoti session and loading data

In [2]:
# Starting atoti session
from atoti.config import create_config
config = create_config(metadata_db="./metadata.db")
session = tt.create_session(config=config)

# Let's load historical data into a store from a csv file.
# Read more about the supported input data sources on the https://docs.atoti.io/.
# By setting array_sep, atoti will recognize the input data is in vectorized format.

market_data_store = session.read_csv(
    "stock_price_returns.csv",
    keys=["Stock"],
    store_name="StockData",
    array_sep=";"
)

market_data_store.head(3)

,Returns,Prices
Stock,,
APA,"doubleVector[3634]{-0.002821133740173898, ...}","doubleVector[3634]{58.06625747680664, ...}"
CCL,"doubleVector[3634]{-0.0038484012008623836, ...}","doubleVector[3634]{36.2109375, ...}"
EOG,"doubleVector[3634]{0.001013001435978822, ...}","doubleVector[3634]{35.61472702026367, ...}"


In [3]:
# Now let's load the positions.
positions_store = session.read_csv(
    "positions_snapshot.csv",
    keys=["Book", "Stock"],
    store_name="Positions",
    types={"AmountUSD": tt.types.DOUBLE},
)

positions_store.head(3)

Portfolio  AmountUSD
Book    Stock                     
Indices ^DJI         US     5000.0
        ^GSPC        US     5000.0
Oil&Gas APA          US      250.0

In [4]:
positions_store.join(market_data_store)

# Creating an OLAP cube

In [5]:
cube = session.create_cube(positions_store, "Historical VaR Cube", mode="auto")
h = cube.hierarchies
l = cube.levels
m = cube.measures
cube

Let's put Books under Portfolios to create a multi-level hierarchy:

In [6]:
h["Portfolio Structure"] = {
    "Portfolio": l["Portfolio"],
    "Book": l['Book']
}

The default aggregation functions created at this step allow to display position's market values. I'm using the multi-level `Portfolio Structure` to expand data:

In [27]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# URL to atoti UI

In [8]:
session.url

'http://localhost:56799'

# New measure for the VaR vector

In our toy example we multiply the positions market values -  _AmountUSD_  - by the stock returns for each name to obtain the historical VaR vectors. This step is not necessary if PL vectors are delivered by a risk system. 

In [9]:
m["VaR Vector"] = tt.agg.sum(
    m["AmountUSD.SUM"] * m["Returns.VALUE"], scope=tt.scope.origin(l["Stock"])
)

We can now explore the VaR vectors at stock and book level.

In [10]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Enriching VaR vectors with date labels

So far, VaR vectors are not intuitive. Let's create a hierarchy that will allow to display P&L value by historical date, which is more user friendly.

In [11]:
calendar = pd.date_range(end = '2020-06-09', periods = 3634, freq="B")

In [12]:
cube.create_parameter_hierarchy(
    "Historical Dates", calendar, index_measure="Date Index",
)

In [13]:
# This new measure will pick an element from the VaR vector:
m["PnL at date"] = m["VaR Vector"][m["Date Index"]]
m["PnL at date"].formatter = "DOUBLE[#,##0.#]"
m["Price at date"] = m["Prices.VALUE"][m["Date Index"]]

Let's do some simple verification. In the following Pivot table (filtered for the 4 recent dates), we can see the PnL value for each date scenario.  

For each date, let's drill down to the books and stock. 

In [14]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

Or, for prices (this view is filtered for three most recent dates):

In [15]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Creating Sliding Windows

We want to compute VaRs for different observation windows to see which period would lead to the most conservative VaR for our current portfolio. Such a "worst" window will be a good candidate for the Stressed VaR calibration and will be a focus for further analysis and decomposition.

In [16]:
# Lets create a measure for the observation window length:
m['Lookback Window'] = 260

In [17]:
m["VaR Sub-Vector"] = m["VaR Vector"][
    m["Date Index"] - m['Lookback Window'] : m["Date Index"]
]

The Sub-Vector measure requires having historical dates in the view. 

Now for each desired scope, the `VaR Sub-Vector` measure will pick 260 most recent values from the `VaR vector`. The below view is filtered for the 3 most recent historical dates. This result is not for human consumption, but still showing it here for illustration purposes.

In [18]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Adding Sliding VaR measure

In this notebook, we'll apply a simple quantile function to the subvector. 

Please read about about **VaR-type estimators using kernel functions** in our future blog posts.

In [19]:
# Hui Fang:
# h["Historical Dates"].slicing = False
# m["Worst VaR"] = tt.agg.min(m["1d 99% VaR Sliding"], scope=tt.scope.siblings(l["Historical Dates"]))
# m["Worst VaR Date Index"] = tt.agg.max(tt.where(m["1d 99% VaR Sliding"] == m["Worst VaR"], m["Date Index"], 0))
# m["max label"] = tt.agg.max_member(m["Worst VaR Date Index"], l["Historical Dates"])

In [20]:
m["1d 99% VaR Sliding"] = tt.array.quantile(m["VaR Sub-Vector"], 0.01)

As expected, VaR chart is step-wise and goes up and down every time extreme scenarios leave the observation window.

In [21]:
# This doesn't work yet:
m["1d 99% VaR Current"] = tt.array.quantile(m["VaR Vector"][-260:], 0.01)

In [23]:
h["Historical Dates"].slicing = False

In [24]:
m['Worst VaR'] = tt.agg.min(m["1d 99% VaR Sliding"],scope=tt.scope.origin(l['Historical Dates']))

In [25]:
cube.query(m['Worst VaR'])

,Worst VaR
0,-1680.811084


And this measure will return the worst scenario:

In [26]:
# the measure should compute the worst sliding window - doesn't work as desired unfortunately
m['Worst VaR Period End'] = tt.agg.min_member(m["1d 99% VaR Sliding"],l['Historical Dates'])

In [28]:
cube.query(m['Worst VaR Period End'])

,Worst VaR Period End
0,2009-04-22


In [29]:
cube.query(m['1d 99% VaR Sliding'])

RuntimeError: ('Request failed', {'status': 'error', 'error': {'errorChain': [{'type': 'IllegalArgumentException', 'message': 'Empty list of fields to retrieve from the store'}, {'type': 'IllegalArgumentException', 'message': 'java.lang.IllegalArgumentException: Empty list of fields to retrieve from the store'}, {'type': 'QueryException', 'message': 'Could not execute query at epoch 3. : ActivePivotSyncActionQuery: action=MdxAction (mdx=SELECT [Measures].[1d 99% VaR Sliding] ON COLUMNS FROM [Historical VaR Cube]), argument=com.quartetfs.pivot.mdx.impl.BasicSelectExecutor@7152bd07'}, {'type': 'MdxException', 'message': 'Error with the following query: SELECT [Measures].[1d 99% VaR Sliding] ON COLUMNS FROM [Historical VaR Cube]'}, {'type': 'BadArgumentException', 'message': '[400] com.quartetfs.pivot.mdx.MdxException: Error with the following query: SELECT [Measures].[1d 99% VaR Sliding] ON COLUMNS FROM [Historical VaR Cube]'}], 'stackTrace': 'com.qfs.fwk.services.BadArgumentException: [

In [ ]:
cube.visualize()

In [ ]:
cube.visualize()

# Filter and Drill down to the worst scenarios

In [ ]:
cube.visualize()

# What-If on different lookback setting

Now let's setup a what-if simulation and change the Lookback Window to shorter and longer lengths. 

In [ ]:
lookback_windows = cube.setup_simulation(
    "Lookback Windows", replace=[m["Lookback Window"]], base_scenario="260 days"
).scenarios
lookback_windows["10 days"] = 10
lookback_windows["120 days"] = 120
lookback_windows["2600 days"] = 2600

Now the different `Lookback Windows` are available in the UI for analysis. As expected, the shorter the observation window, the faster VaR metric reacts to changes in market volatility.

In [ ]:
cube.visualize()